# Preparation

In [1]:
# connect to my Google Drive
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/FAES/BIOF475_83_Intro_to_DS/wk1/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# print work directory
os.getcwd()

'/content/drive/My Drive/FAES/BIOF475_83_Intro_to_DS/wk1'

In [3]:
# check available files
! ls

Homework1_Yijun.ipynb  post-operative_clean.csv  post-operative.data  post-operative.names


In [4]:
# Import the libraries
import pandas as pd
import numpy as np

# (1) Load the post-operative.data CSV using Pandas

In [5]:
df_data = pd.read_csv("post-operative.data", header=0)
df_data.head()

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,ADM-DECS
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


# (2) Drops all rows that contain missing data (indicated by ’?’)



In [6]:
# find rows that contain missing data
missing_rows = df_data[(df_data == '?').any(axis=1)].index
missing_rows

Index([46, 48, 70], dtype='int64')

In [7]:
# get a dataframe that contains the '?' in a row
df_data.iloc[df_data[(df_data == '?').any(axis=1)].index]

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,ADM-DECS
46,low,mid,good,high,unstable,stable,stable,?,I
48,high,high,excellent,high,stable,stable,unstable,?,A
70,mid,low,excellent,mid,unstable,unstable,unstable,?,A


In [8]:
# replace '?' values with NaN
df_data.replace(to_replace='?', value=np.nan, inplace=True)

In [9]:
# drop rows that contain a NaN value
df_data.dropna(inplace=True)

In [10]:
# check if the dataframe still contains the '?' in rows
df_data.iloc[df_data[(df_data == '?').any(axis=1)].index]

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,ADM-DECS


# (3) Replaces all binary variables with indicator variables

In [11]:
# count the unique entries of each column
df_data.nunique()

,0
L-CORE,3
L-SURF,3
L-O2,2
L-BP,3
SURF-STBL,2
CORE-STBL,3
BP-STBL,3
COMFORT,4
ADM-DECS,3


It seems like two of the columns "L-O2" and "SURF-STBL" are binary variables:

In [12]:
df_data["L-O2"].unique()

array(['excellent', 'good'], dtype=object)

In [13]:
df_data["SURF-STBL"].unique()

array(['stable', 'unstable'], dtype=object)

## Convert the binary variable of "L-O2" to an indicator variable




In [14]:
df_data = pd.concat([df_data,pd.get_dummies(df_data["L-O2"], drop_first=True, prefix="L-O2",dtype=int)],axis=1)
df_data.drop(["L-O2"], axis=1, inplace=True)
df_data.head()

,L-CORE,L-SURF,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,ADM-DECS,L-O2_good
0,mid,low,mid,stable,stable,stable,15,A,0
1,mid,high,high,stable,stable,stable,10,S,0
2,high,low,high,stable,stable,mod-stable,10,A,0
3,mid,low,high,stable,unstable,mod-stable,15,A,1
4,mid,mid,high,stable,stable,stable,10,A,0


## Convert the binary variable of "SURF-STBL" to an indicator variable

In [15]:
df_data = pd.concat([df_data,pd.get_dummies(df_data["SURF-STBL"], drop_first=True, prefix="SURF-STBL",dtype=int)],axis=1)
df_data.drop(["SURF-STBL"], axis=1, inplace=True)
df_data.head()

,L-CORE,L-SURF,L-BP,CORE-STBL,BP-STBL,COMFORT,ADM-DECS,L-O2_good,SURF-STBL_unstable
0,mid,low,mid,stable,stable,15,A,0,0
1,mid,high,high,stable,stable,10,S,0,0
2,high,low,high,stable,mod-stable,10,A,0,0
3,mid,low,high,unstable,mod-stable,15,A,1,0
4,mid,mid,high,stable,stable,10,A,0,0


# (4) Replaces all categorical variables with more than two values with indicator vectors

In [16]:
# count the unique entries of each column
df_data.nunique()

,0
L-CORE,3
L-SURF,3
L-BP,3
CORE-STBL,3
BP-STBL,3
COMFORT,4
ADM-DECS,3
L-O2_good,2
SURF-STBL_unstable,2


In [17]:
# replace columns other than "L-O2_good" and "SURF-STBL_unstable" with indicator variables
for col in df_data.columns:
  if col not in ["L-O2_good", "SURF-STBL_unstable"]:
    df_data = pd.concat([df_data,pd.get_dummies(df_data[col], drop_first=True, prefix=col,dtype=int)],axis=1)
    df_data.drop([col], axis=1, inplace=True)

df_data.head()

,L-O2_good,SURF-STBL_unstable,L-CORE_low,L-CORE_mid,L-SURF_low,L-SURF_mid,L-BP_low,L-BP_mid,CORE-STBL_stable,CORE-STBL_unstable,BP-STBL_stable,BP-STBL_unstable,COMFORT_07,COMFORT_10,COMFORT_15,ADM-DECS_I,ADM-DECS_S
0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1
2,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0
3,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0
4,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0


# (5) Saves the resulting dataframe as a CSV, making sure not to include the index column

In [18]:
df_data.to_csv("post-operative_clean_Yijun.csv", index=False)